In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [28]:
import matplotlib as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif

In [3]:
def make_df(directory):
    csv = pd.read_csv(directory)
    df = pd.DataFrame(csv)
    return df

In [4]:
df_train = make_df('/kaggle/input/spaceship-titanic/train.csv')
df_test = make_df('/kaggle/input/spaceship-titanic/test.csv')

In [6]:
df_train.shape

(8693, 14)

In [5]:
df_train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [40]:
df_test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [39]:
pd.set_option('display.max_rows', 20)
df_train

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,Spa,VRDeck,Name,Transported,Pri_cabin,Sub_cabin,Cabin_num,Service_value
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,Maham Ofracculy,False,B,0,P,0.0
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,549.0,44.0,Juanna Vines,True,F,0,S,143.0
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,6715.0,49.0,Altark Susent,False,A,0,S,3619.0
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,3329.0,193.0,Solam Susent,False,A,0,S,1654.0
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,565.0,2.0,Willy Santantines,True,F,1,S,524.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,55 Cancri e,41.0,True,1643.0,74.0,Gravior Noxnuther,False,A,98,P,6819.0
8689,9278_01,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,Kurta Mondalley,False,G,1499,S,0.0
8690,9279_01,Earth,False,TRAPPIST-1e,26.0,False,1.0,0.0,Fayey Connon,True,G,1500,S,1872.0
8691,9280_01,Europa,False,55 Cancri e,32.0,False,353.0,3235.0,Celeon Hontichre,False,E,608,S,1049.0


In [32]:
df_train_NaDrop = df_train.dropna(axis=0)

In [33]:
def factorize_obj(df):
    for i in df.select_dtypes('object'):
        df[i], _ = df[i].factorize()
    return df

In [34]:
def mu_info(df):
    y = df['Transported']
    X = df.copy()
    X = factorize_obj(X)
    info = mutual_info_classif(X, y)

    for i, feature in enumerate(X.columns):
        print(f'Feature: {feature}, MI: {info[i]}')

In [35]:
mu_info(df_train_NaDrop)

Feature: PassengerId, MI: 0.006405750470873439
Feature: HomePlanet, MI: 0.015662650061314842
Feature: CryoSleep, MI: 0.11659023184233996
Feature: Destination, MI: 0.0036335396966495903
Feature: Age, MI: 0.015893355476858995
Feature: VIP, MI: 0.01313265252966489
Feature: Spa, MI: 0.07062844710081873
Feature: VRDeck, MI: 0.06081906293680395
Feature: Name, MI: 0.007544632367977311
Feature: Transported, MI: 0.6940290530079508
Feature: Pri_cabin, MI: 0.030314856773520305
Feature: Sub_cabin, MI: 0.00938948133410622
Feature: Cabin_num, MI: 0.011657804469159538
Feature: Service_value, MI: 0.12691603158558618


# D_set ver1

expand and consolidate data

In [16]:
def separate_cabin(df):
    df[['Pri_cabin', 'Sub_cabin', 'Cabin_num']] = df['Cabin'].str.split('/', expand=True, n=3)
    return df

In [18]:
df_train_cabintest = separate_cabin(df_train)

df_train_cabintest['Service_value'] = df_train_cabintest['RoomService'] + df_train_cabintest['FoodCourt'] + df_train_cabintest['ShoppingMall']

In [20]:
df_train_cabintest.drop(['Cabin', 'RoomService', 'FoodCourt', 'ShoppingMall'], axis=1, inplace=True)

In [21]:
df_train_cabintest

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,Spa,VRDeck,Name,Transported,Pri_cabin,Sub_cabin,Cabin_num,Service_value
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,Maham Ofracculy,False,B,0,P,0.0
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,549.0,44.0,Juanna Vines,True,F,0,S,143.0
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,6715.0,49.0,Altark Susent,False,A,0,S,3619.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8690,9279_01,Earth,False,TRAPPIST-1e,26.0,False,1.0,0.0,Fayey Connon,True,G,1500,S,1872.0
8691,9280_01,Europa,False,55 Cancri e,32.0,False,353.0,3235.0,Celeon Hontichre,False,E,608,S,1049.0
8692,9280_02,Europa,False,TRAPPIST-1e,44.0,False,0.0,12.0,Propsh Hontichre,True,E,608,S,4814.0
